In [86]:
import pandas as pd
import numpy as np
import networkx as nx



In [87]:
def get_sortednodes(graph, weight_df):
    c = 0
    node_w_dict = {'node':[], 'w':[]}
    for node in graph.nodes():
        node_acc = 0
        c = c + 1
        #nborlist = nx.classes.function.all_neighbors(G,node)
        
        #for nbor in nborlist:
            
            #nbor_weight = G.get_edge_data(node,nbor)
            
            #this = weight_df.sort_values([node,'w'],ascending=False).groupby('w')
            #print(this)
            #weight_list = [v['w'].iloc[0] for k, v in this if k==node]
            #edge_weight = weight_df.loc[weight_df['u']==node, 'w']
            
            
        nodeW = get_node_weight(weight_df,node )
        
        node_acc = nodeW + node_acc
        #print(node_acc)
        node_w_dict['node'].append(node)
        node_w_dict['w'].append(round(node_acc,2))
        if c==2:
            break
        
        
    node_w_df = pd.DataFrame.from_dict(node_w_dict)

    return node_w_df.sort_values(by=['w'],ascending=False)




In [88]:
# clustering 


def get_edgew(graph):
    
    
    #print('This node',this_node, 'has these many neighbours: ',len(list(this_node_nbors)))
    #print()
    #cn_nbors = nx.classes.function.all_neighbors(cn,max_node[0])
    
    
    edge_weight = {'u':[], 'v':[], 'w':[]}
    
    for node in graph.nodes():
        this_node_nbors = nx.classes.function.all_neighbors(graph,node)
        for nbor in list(this_node_nbors):
            wt = 0
            
            wt = len(list(nx.common_neighbors(graph,node, nbor)))
            #print(node, nbor, wt)

            edge_weight['u'].append(node)
            edge_weight['v'].append(nbor)
            edge_weight['w'].append(wt)
        this_node_nbors = None
        
      
    edge_weight = pd.DataFrame.from_dict(edge_weight)
    return edge_weight
    #return edge_weight.sort_values(by=['w'],ascending=False)   
   

In [89]:
def get_node_weight(w_df, n):
    accm_w = 0 
    for idx, row in w_df.iterrows():
        if row['u']==n:
            #print('node ---',row['u'], 'weight ----- ',row['w'])
            accm_w = accm_w + row['w']
            
    return accm_w
   

In [90]:
def get_sum_nbor(candidate, Graph):
        
        accm_w = 0
        for nodes in Graph.nodes():
            nbors = len(list(nx.common_neighbors(G,candidate,node)))
            accm_w = accm_w + nbors
        return accm_w

In [91]:


def total_edges(cluster, graph, e_candidate):
    accm_edges = 0
    for node in cluster:
        edges = graph.number_of_edges(node, e_candidate)
        accm_edges = edges + accm_edges
       
    return accm_edges 


In [92]:
def find_v(cl_n, e_wt):
    for idx, rows in e_wt.iterrows():
        if cl_n == rows['u']:
            #print(idx)
            return e_wt.iloc[idx]

In [93]:

def cluster_add(cluster,Graph, cluster_nodes_sorted, edge_weights):
    
    c=0
    # start with sorted cluster nodes
    for idx, cl_node in cluster_nodes_sorted.iterrows():
        
        
        
        
        cl_node = cl_node['node']
        
        #get cluster node neighbors in GRAPH
        nborlist = nx.classes.function.all_neighbors(Graph,cl_node)

        edge_weight = {'u':[], 'v':[], 'w':[]}

        #for each nbor in cluster Node
        for nbor in nborlist:
            nbr_weight = 0
            #print(nbor)
            
            #chech if this neighbor is in the cluster or not
            if nbor not in cluster_nodes_sorted:
                #print('True')
                
                # for every node in the cluster nodes sorted DF
                for idx, row in edge_weights.iterrows():
                    #check for this nbor and cluster weights in the graph weights table edge_weights
                    if row['u'] == cl_node and row['v']== nbor:
                        c = c + 1
                        if c==2:
                            edge_weight = pd.DataFrame.from_dict(edge_weight)
                            edge_weight = edge_weight.sort_values(by=['w'],ascending=False)
                            return edge_weight
                        print(' row form graph',row['u'], row['v'])
                        print( 'loop cluster node ', cl_node,'nbor in loop: ',nbor)
                        nbr_weight = nbr_weight + row['w']

                        edge_weight['u'].append(cl_node)
                        edge_weight['v'].append(nbor)
                        edge_weight['w'].append(nbr_weight)
    edge_weight = pd.DataFrame.from_dict(edge_weight)
    edge_weight = edge_weight.sort_values(by=['w'],ascending=False)
    return edge_weight

In [99]:

def add_nbor_to_cluster(nbor_from_graph):
    u , v , w = edge_from_cluster.iloc[0]
    cluster.add_edges_from([(u,v,{'w':w})])


    density = nx.classes.function.density(cluster)
    number_of_nodes = nx.classes.function.number_of_nodes(cluster)


    print(number_of_nodes, density)
    edges_total = w
    cp = edges_total /(number_of_nodes * density)
    
    if cp<0.2 and density<0.6:
        cluster.remove_edges_from([(u,v,{'w':w})])
        cluster.remove_node(v)

def add_cluster_to_list(cluster, cluster_list):
    cluster_list.append(cluster)
    return cluster_list
    
    
    
    

In [97]:
##################------------------MAIN--------------------####################



def main_f():

    #Read graph into Pandas Dataframe
    df = pd.read_csv('file.csv', sep="\t", names=['u','v','w'])

    #read Graph from Pandas Dataframe into Netoworkx Graph

    G = nx.from_pandas_edgelist(df,'u','v','w')

    #density and cp threshold
    density = 0.6
    cp = 0.2

    #list of clusters from Graph
    cluster_list = []

    #Get edges weights of this graph
    # WARNING !!!!!!!!!!!!!!! TAKES A LONG TIME CHECK THIS FUNCTION CODE TO REMOVE BREAKS BEFORE REMOVING COMMENT
    edge_weights = get_edgew(G)
    #edge_weights


    # WARNING HERE !!!!!! TAKES A LONG TIME 
    sorted_nodes = get_sortednodes(G, edge_weights)
    #sorted_nodes


    #get nodes sorted by weight from Graph
    #sorted_node_df = get_sortednodes(,G)

    #MAKE A NEW CLUSTER 
    cluster = nx.Graph()

    #ADD FIRST TOP NODE TO CLUSTER ACCORDING TO SPECS IN ALGO
    #cl_node = sorted_nodes.iloc[0]['node']
    
    #for each of the top nodes in the Graph 
    # add them to cluster 
    i=0
    for idx, row in sorted_nodes.iterrows():
        if i==1:
            break
        cl_node = row['node']
        # add DESTINATION "V" OF THE FIRST NODE 
        cl_vw = find_v(cl_node, edge_weights)
        u,v,w = cl_vw[0],cl_vw[1],cl_vw[2]
        print(u,v,w)
        cluster.add_edges_from([(u,v,{'w':w})])

        # sort nodes in cluster and get their sorted edges and sorted top nodes 
        cluster_edge_weight = get_edgew(cluster)
        cluster_nodes_sorted  = get_sortednodes(cluster, cluster_edge_weight)
        #cluster.nodes()

        #GET NEIHBORS FROM GRAPH AND ADD THEM TO CLUSTER ACCORDING TO ALGO FLOW
        nbor_from_graph = cluster_add(cluster,G, cluster_nodes_sorted,edge_weights)
        #ADD THIS CLUSTER TO CLUSTER LIST
        cluster_list = add_cluster_to_list(cluster, cluster_list)
        #delete this cluster 
        cluster = None
        i=i+1
    
    return cluster_list

In [100]:
# RUN THIS ALGO


myclusters = []

myclusters = main_f()

TFC3 MYO4 208
 row form graph TFC3 MYO4
loop cluster node  TFC3 nbor in loop:  MYO4


In [103]:
myclusters[0].nodes()

NodeView(('TFC3', 'MYO4'))

In [12]:
# WARNING !!!!!!!!!!!!!!!
#edge_weights = get_edgew(G)


edge_weights

#edge_weights.sort_values(by=['w'],ascending=False).iloc[0]['v']

,u,v,w
0,TFC3,MYO4,208
1,TFC3,SHP1,265
2,TFC3,TFC1,112
3,TFC3,"DUR1,2",68
4,TFC3,NOP1,48
...,...,...,...
513909,YLL047W,RSC4,0
513910,YIL060W,GCN3,0
513911,SNO2,TFA2,0
513912,YGL262W,PXL1,0


In [17]:
#sorted_nodes = get_sortednodes(G, edge_weights)

sorted_nodes

,node,w
2,SHP1,138272
23,HTA2,93606
0,TFC3,63004
9,YRA1,54248
1,MYO4,50774
20,LTE1,50316
22,HTB2,46252
28,LYS2,29342
3,TFC1,21966
26,HHF1,17470


In [29]:

cluster = None
cluster = nx.Graph()
cl_node = sorted_nodes.iloc[0]['node']


cl_vw = find_v(cl_node, edge_weights)
u,v,w = cl_vw[0],cl_vw[1],cl_vw[2]
print(u,v,w)


# add first node from graph 
cluster.add_edges_from([(u,v,{'w':w})])


#cluster.add_node('SHP1')







SHP1 TFC3 265


In [36]:



# sort nodes in cluster and get their sorted edges and sorted top nodes 
cluster_edge_weight = get_edgew(cluster)

cluster_nodes_sorted  = get_sortednodes(cluster, cluster_edge_weight)
#cluster.nodes()



NodeView(('SHP1', 'TFC3'))

In [39]:
cluster_edge_weight

,u,v,w
0,SHP1,TFC3,0
1,TFC3,SHP1,0


In [38]:

#cluster.nodes()

cluster_nodes_sorted.head()

,node,w
0,SHP1,0
1,TFC3,0


In [74]:

nbor_from_graph = cluster_add(cluster,G, cluster_nodes_sorted,edge_weights)

cluster_list = add_cluster_to_list(cluster, cluster_list)

#nbr_to_add = pd.DataFrame.from_dict(edge_weight)
#add_this_nbr = nbr_to_add.sort_values(by=['w'],ascending=False).iloc[0]

 row form graph SHP1 TFC3
loop cluster node  SHP1 nbor in loop:  TFC3


In [415]:
n = sorted_nodes.iloc[0]
print(n['node'])

TFC3


In [431]:
g=0
for idx, row in edge_weights.iterrows():
    if row['v'] == 'MYO4':
        g = g + row['w']
g
#here['v'=='MYO4']

#wt = G.get_edge_data('TFC3','MYO4')
#wt['w']

50774

In [401]:
#edge_weights.drop_duplicates(subset=['u'])
#edge_weights

#this = edge_weights.sort_values(['w','u'],ascending=False).groupby('u')
#type(this)
nodeW = get_node_weight(edge_weights, 'SHP1')
#nodeW
#this.head()

nodeW

138272

In [374]:
#this = edge_weights.loc[edge_weights['u']=='SPT16', edge_weights['v']=='COX1', 'w']
#this = edge_weights.groupby(['u','v'])

#node = 
this = edge_weights.sort_values(['w'],ascending=False).groupby('u')
#this = edge_weights.groupby('u')
#print(this.head())
this.head()

,u,v,w
338345,YJL109C,HCA4,1043
58684,HCA4,YJL109C,1043
96471,SPT16,HCA4,943
57896,HCA4,SPT16,943
96490,SPT16,YJL109C,851
...,...,...,...
364566,MIG2,ASK1,0
365740,SPP2,CDC13,0
365598,MET31,DAD1,0
365597,MET31,RRI1,0


In [286]:
for k, v in this:
    if k =='SPT16':
        print(v['w'].iloc[0:9])

4223    1672
5907    1672
5905    1672
4226    1672
4225    1672
4224    1672
4222    1672
4221    1672
4220    1672
Name: w, dtype: int64


In [361]:
mynode = nx.classes.function.all_neighbors(G,'YJL109C')
list(mynode)

['RPN6',
 'YDR101C',
 'YDR324C',
 'SMB1',
 'ASN2',
 'YGR128C',
 'RPN10',
 'Q0065',
 'IMD1',
 'HTB2',
 'HTA2',
 'STU1',
 'SSA3',
 'HHF1',
 'NHP6B',
 'GRS1',
 'ROT2',
 'AGP1',
 'RRP7',
 'SPB1',
 'CHA1',
 'YCR024C',
 'RDS1',
 'NOP1',
 'YDL085CA',
 'SAS10',
 'MAK21',
 'SEC1',
 'HMO1',
 'COX20',
 'ATP5',
 'YDR306C',
 'SWR1',
 'YRA1',
 'YDR398W',
 'RPN9',
 'POL5',
 'KRE31',
 'FLO8',
 'RPS26B',
 'YER152C',
 'BST1',
 'PMA1',
 'SCL1',
 'PUF4',
 'SNT2',
 'SNU71',
 'DBF2',
 'NOP7',
 'CLC1',
 'UBR1',
 'PDX1',
 'CCH1',
 'ORC6',
 'YHR196W',
 'SER33',
 'PRK1',
 'RPL40A',
 'SGN1',
 'URA2',
 'MPP10',
 'SFT1',
 'LOS1',
 'YKR046C',
 'EMP70',
 'STM1',
 'CBF5',
 'SIK1',
 'MEC3',
 'VIP1',
 'CNA1',
 'APT1',
 'SPC2',
 'ATR1',
 'NAM7',
 'YMR093W',
 'YPK2',
 'YMR124W',
 'CEP3',
 'TRS130',
 'YMR259C',
 'BUL1',
 'HEF3',
 'YNL054WB',
 'RPA49',
 'MCK1',
 'YNR009W',
 'YNR014W',
 'UFE1',
 'MGM1',
 'RRS1',
 'YOR385W',
 'RRP12',
 'RAD1',
 'NAN1',
 'YPL146C',
 'OYE3',
 'RPL36B',
 'DIM1',
 'YME1',
 'BRE2',
 'SMD2',
 'HSC

In [ ]:

node = ['TFC3']
this_node_nbors = nx.classes.function.all_neighbors(Graph,node)

for nbor in this_node_nbors:
    if 
print(this_node_nbors)


In [185]:

#edge_weights = get_edgew(G)
#sorted_nodes = get_sortednodes(G, edge_weights)

for idx, node in sorted_nodes.iterrows():
    print(node)
    
    #cluster.add_node(node)
    
    for node in cluster.nodes():
        get_sum_w = total_edges(cluster, Graph, )
    break


KeyboardInterrupt: 

In [177]:








# got through each node in sorted nodes from Graph
for idx, node in sorted_node_df.iterrows():
    
    
    
    if idx==2:
        break
    
    #Make new Graph / Cluster
    
    # Add Current Node to cluster
    # this_node contains STRING NAME
    this_node = node['node']
    print('this node',this_node)
    cluster.add_node(this_node)
    
    # Add Top neighbor of this node with heighest Weight metric, i.e. most common neighbors
    
    topnbor = get_edgew(G)
    
    print(type(topnbor))
    #edge_sum = total_edges(cluster, G, topnbor['v'])
    
    for idx, nbors in topnbor.iterrows():
        print(idx,nbors['v'],nbors['w'])
        #cluster.add_node(nbors['v'])
        #cluster.add_edges_from([(this_node,top_nbor['v'],{'w':node['w']})])
    
    #density = nx.classes.function.density(cluster)
    #number_of_nodes = nx.classes.function.number_of_nodes(cluster)
    
    #print(number_of_nodes, density)
    #edges_total = total_edges(cluster,G,node['v'])
    #cp = edges_total /(number_of_nodes * density)
    #if cp<0.2 and density<0.6:
    #    cluster.remove_edges_from([(node,cluster_node['v'],{'w':node['w']})])
     #   cluster.remove_node(cluster_node['v'])
    
    

print(nx.classes.function.number_of_nodes(cluster))    


SyntaxError: invalid syntax (<ipython-input-177-c2513c02a3e8>, line 21)

In [98]:
cluster = get_sortednodes(G)
#print(type(cluster))

acc_w = 0
for idx, node in cluster.iterrows():
    #print(node['node'])
    this = node['node']
    top_nbr = get_nbor(this)
    print(top_nbr['w'])
    
    acc_w = acc_w + get_nbor(this)['w']
    if idx == 5:
        break
print(acc_w)

418
748
365
105
943
102
2681


In [9]:
cluster_list[].nodes()



NodeView(('YRA1', 'TFC3', 'HTA2', 'ERP2', 'RVS167', 'YAL027W', 'CRN1', 'TPD3', 'SPT16', 'HTB2', 'NOP1', 'RPN3', 'SWD3', 'STM1', 'HSC82', 'YBR030W', 'HMO1', 'SEM1', 'UBP14', 'HCA4', 'PTA1', 'SGV1', 'HHF1', 'RPN5', 'HTL1', 'RPN1', 'UBP6', 'OSH1', 'CKA1', 'RPN6', 'REB1', 'NAP1', 'DRS2', 'SRB5', 'SRB4', 'MLC1', 'CDC34', 'RAD23', 'KTI11', 'RRP5', 'PEP1', 'YGR090W', 'MYO4', 'MAM33', 'NCL1', 'HHT1', 'PIL1', 'RPT6', 'LYS20', 'RRP45', 'RRP46', 'RPL40A', 'NOP7', 'GLT1', 'YJL109C', 'RPN2', 'RPT2', 'HAT1', 'ARP4', 'FUN36', 'SRP1', 'NUP60', 'RRS1', 'RPG1', 'PHO11', 'SWD2', 'RPN9', 'CTR86', 'YDR365C', 'QRI2', 'MTR4', 'CDC48', 'SHP1', 'MRPS5', 'RPB9', 'RPB5', 'KAP95', 'KEM1', 'BUD23', 'RPN11', 'SIK1', 'OAF1', 'SSB2', 'IWR1', 'ROX3', 'RPN8', 'RSC4', 'PRP8', 'RPB1', 'RFA1', 'STH1', 'BMH2', 'RSC8', 'HHT2', 'FUN14', 'NAT1', 'ASF1', 'TAF6', 'PRT1', 'YEF3', 'CBF5', 'RVB1', 'TIF2', 'EFT2', 'AAR2', 'YHR121W', 'CCR4', 'PRP43', 'BEM2', 'RPB3', 'YPL004C', 'SEC28', 'COP1', 'RPB10', 'CMD1', 'IES1', 'SUP35', 'RPF2

In [409]:
cluster_node = get_nbor(firstnode)
cluster_node['w']

G.add_edges_from([(2, 3, {'weight': 8})])

170

In [92]:


def sumofweight( Graph, candidate)

sum_of_w = 0


for this_node in Graph.nodes():
    
    
    
    nbors = 0
    #print(candidate)
    nbors = len(list(nx.common_neighbors(Graph,candidate,this_node)))
    nbors = nbors + nbors
    
    #print(nbors)
    
    return 
        #return edge_weight.sort_values(by=['w'],ascending=False).iloc[0]

    top  = get_nbor(node)
    #print(top['w']) 
    
    sum_of_w = sum_of_w + top['w']
print(sum_of_w)    

In [405]:
len(list(nx.common_neighbors(G,'YRA1','TFC3')))

170

In [380]:
firstnode[0]

firstnode_nbors = nx.classes.function.all_neighbors(G,firstnode[0])
type(firstnode_nbors)
#print(len(list(firstnode_nbors)))
for nbor in list(firstnode_nbors):
    #print('here-----------------',nbor)
    candidate_n = nx.classes.function.all_neighbors(G,nbor)
    #print(list(candidate_n))
    candidate_list = list(candidate_n)
    firstnode_list = list(firstnode_nbors)
    edge_weight = sum(np.isin(list(candidate_list),list(firstnode_list)))
    print('EW ++++++',edge_weight)
#candidate_n = nx.classes.function.all_neighbors(G,firstnode)


#edge_weight_temp = sum(np.isin(list(first_node_nbors),list(candidate_n)[0]))



EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW ++++++ 0
EW +

In [312]:
first_node_nbors = nx.classes.function.all_neighbors(G,firstnode['node'])




testlist = [first_node_nbors[n] for n in first_node_nbors]

#nbor_node = [nx.classes.function.all_neighbors(G,list(first_node_nbors)[n]) for n in first_node_nbors]
#print(nbor_node)
#print(len(list(first_node_nbors)))
#print(len(list(nbor_node)))
#mask = sum(np.isin(nbor_node,list(first_node_nbors)))
#mask

TypeError: 'dict_keyiterator' object is not subscriptable

0

In [102]:
G.edges(data=True)
heaviest_node = sorted(G.edges(data=True), key= lambda x: x[2]['w'], reverse=True)[0]
print(heaviest_node[0])

TFC3


In [153]:
nlist = list(nx.classes.function.all_neighbors(G,"DUR1,2"))
nacc = 0
for nbor in nlist:
        nbor_weight = G.get_edge_data("DUR1,2",nbor)
        nacc = nbor_weight['w'] + nacc
        print(nacc)

0.015
0.027
0.048
0.062
0.176
0.191
0.20600000000000002
0.22100000000000003
0.23500000000000004
0.25000000000000006
0.26500000000000007
0.2790000000000001
0.39300000000000007
0.4110000000000001
0.4300000000000001
0.4440000000000001
0.45800000000000013
0.47700000000000015
0.49600000000000016
0.5150000000000001
0.5270000000000001
0.5390000000000001
0.5530000000000002
0.5680000000000002
0.5820000000000002
0.5970000000000002
0.6110000000000002
0.6260000000000002
0.6400000000000002
0.6550000000000002
0.6700000000000003
0.6840000000000003
0.6990000000000003
0.7140000000000003
0.7290000000000003
0.7430000000000003
0.7580000000000003
0.7720000000000004
0.7860000000000004
0.8010000000000004
0.8160000000000004
0.8310000000000004
0.8460000000000004
0.8610000000000004
0.8760000000000004
0.8910000000000005
0.9050000000000005
0.9190000000000005
0.9340000000000005
0.9480000000000005
0.9630000000000005
0.9770000000000005
0.9920000000000005
1.0070000000000006
1.0210000000000006
1.0360000000000005
1.050

In [ ]:
max_w = 0
for u,v,w in G:
    heaviest_w  = len(nx.common_neighbors(G,u,v))
    if max_w < heaviest_w:
        max_w = heaviest_w
    

In [234]:
#matrix = nx.attr_matrix(G, edge_attr='w', rc_order=None)[0]
#matrix

#weight_matrix = matrix * matrix
#weight_matrix.shape

Gx = nx.Graph()
Gx.add_edge('A','C',weight=1)
Gx.add_edge('A','D', weight=4)
Gx.add_edge('C','D', weight=3)

matrix = nx.attr_matrix(Gx, edge_attr='weight', rc_order=['A','C','D'])

matrix
#matrix = matrix * matrix
#print(matrix)

matrix([[0., 1., 4.],
        [1., 0., 3.],
        [4., 3., 0.]])